In [2]:
from agent.dspy_agent import OpenBBAgentChroma
from agent.database import load_database, build_database, build_docs_metadata, build_graph
from dotenv import load_dotenv,find_dotenv
import os
load_dotenv(find_dotenv(),override=True)

True

In [3]:
openbb_graph,router_names = build_graph()
docs,metadata = build_docs_metadata(openbb_graph)

In [4]:
len(docs),len(metadata)

(337, 337)

In [5]:
# build_database(docs,metadata,os.environ['OPENAI_API_KEY'])

In [6]:
openbb_collection = load_database(os.environ['OPENAI_API_KEY'])

In [7]:
obb_chroma = OpenBBAgentChroma(openbb_collection)

In [10]:
funcs,prompts = obb_chroma("What is the cryptocurrency pair of Ethereum")

First level string: crypto: Get historical price data for cryptocurrency pair(s) within a provider. Search available cryptocurrency pairs within a provider. 

currency: Currency Historical Price. Currency historical data.  Currency historical prices refer to the past exchange rates of one currency against another over a specific period. This data provides insight into the fluctuations and trends in the foreign exchange market, helping analysts, traders, and economists understand currency performance, evaluate economic health, and make predictions about future movements. Currency Search.  Search available currency pairs. Currency pairs are the national currencies from two countries coupled for trading on the foreign exchange (FX) marketplace. Both currencies will have exchange rates on which the trade will have its position basis. All trading within the forex market, whether selling, buying, or trading, will take place through currency pairs. (ref: Investopedia) Major currency pairs inc

In [11]:
funcs

[{'ids': ['id152'],
  'embeddings': None,
  'metadatas': [{'function_call': '{\'name\': \'obb-crypto-search_fmp\', \'description\': \'Search available cryptocurrency pairs within a provider.\', \'parameters\': {\'type\': \'object\', \'properties\': {\'query\': {\'type\': \'string\', \'description\': \'Search query.\', \'default\': None}, \'provider\': {\'description\': "The provider to use for the query, by default None. If None, the provider specified in defaults is selected or \'fmp\' if there is no default.", \'default\': \'fmp\', \'type\': \'string\', \'enum\': [\'fmp\']}}, \'required\': [\'provider\']}}',
    'node_name': 'obb-crypto-search_fmp',
    'provider_source': 'fmp',
    'trail': 'crypto-->search',
    'type': 'provider_function'}],
  'documents': ['empty'],
  'uris': None,
  'data': None}]

In [8]:
from openbb import obb
obb.account.login(pat=os.environ['OBB_PAT'])

In [14]:
from typing import List
from agent.function_call import run_function_calling, format_function
def run_main_function_callinng(question:str,provider_list:List[str]=[]):
    funcs,prompts = obb_chroma(question)
    print(funcs)
    provider_sources = [fn['provider_source'] for fn in funcs[0]['metadatas']]
    if provider_list != []:
        valid_providers = [sources for sources in provider_sources if sources in provider_list]
    else:
        valid_providers = provider_sources
    
    return funcs,valid_providers
    # resp = run_function_calling(funcs,valid_providers[0],question)
    # print(resp)
    # final_func = format_function(resp)
    # return final_func

In [15]:
funcs,valid_providers = run_main_function_callinng("I want the futures Gold prices")

First level string: derivatives: Get the complete options chain for a ticker. Historical futures prices. Futures Term Structure, current or historical. 

economy: Get Forecasted GDP Data. Get Nominal GDP Data. Get Real GDP Data. Get the upcoming, or historical, economic calendar of global events. Get Consumer Price Index (CPI).  Returns either the rescaled index value, or a rate of change (inflation). Get Market Risk Premium by country. Search for FRED series or economic releases by ID or string.  This does not return the observation values, only the metadata. Use this function to find series IDs for `fred_series()`. Get data by series ID from FRED. Get Money Measures (M1/M2 and components).  The Federal Reserve publishes as part of the H.6 Release. Get global unemployment data. Use the composite leading indicator (CLI).  It is designed to provide early signals of turning points in business cycles showing fluctuation of the economic activity around its long term potential level.  CLIs 

In [37]:
funcs

[{'ids': ['id163'],
  'embeddings': None,
  'metadatas': [{'function_call': '{\'name\': \'obb-derivatives-futures-historical_yfinance\', \'description\': \'Historical futures prices.\', \'parameters\': {\'type\': \'object\', \'properties\': {\'symbol\': {\'type\': \'string\', \'description\': \'Symbol to get data for. Multiple items allowed for provider(s): yfinance.\', \'default\': \'\'}, \'start_date\': {\'type\': \'string\', \'description\': \'Start date of the data, in YYYY-MM-DD format.\', \'default\': None}, \'end_date\': {\'type\': \'string\', \'description\': \'End date of the data, in YYYY-MM-DD format.\', \'default\': None}, \'expiration\': {\'type\': \'string\', \'description\': \'Future expiry date with format YYYY-MM\', \'default\': None}, \'provider\': {\'description\': "The provider to use for the query, by default None. If None, the provider specified in defaults is selected or \'yfinance\' if there is no default.", \'default\': \'yfinance\', \'type\': \'string\', \'enu

In [38]:
import ast
for func in funcs:
    if func != []:
        for meta in func["metadatas"]:
            if meta["provider_source"] == valid_providers[0]:
                function_call = ast.literal_eval(meta["function_call"])
                function_call["name"] = function_call["name"].rpartition("_")[0]
                break

In [39]:
function_call

{'name': 'obb-derivatives-futures-historical',
 'description': 'Historical futures prices.',
 'parameters': {'type': 'object',
  'properties': {'symbol': {'type': 'string',
    'description': 'Symbol to get data for. Multiple items allowed for provider(s): yfinance.',
    'default': ''},
   'start_date': {'type': 'string',
    'description': 'Start date of the data, in YYYY-MM-DD format.',
    'default': None},
   'end_date': {'type': 'string',
    'description': 'End date of the data, in YYYY-MM-DD format.',
    'default': None},
   'expiration': {'type': 'string',
    'description': 'Future expiry date with format YYYY-MM',
    'default': None},
   'provider': {'description': "The provider to use for the query, by default None. If None, the provider specified in defaults is selected or 'yfinance' if there is no default.",
    'default': 'yfinance',
    'type': 'string',
    'enum': ['yfinance']},
   'interval': {'description': 'Time interval of the data to return.',
    'default': '1

In [62]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([("human", "{input}"),("system","You can write functions from the given tool. Double check your response with correct parameter names and values. Also, check for any invalid parameter values. Also, write a python function to save the output of the code.")])

model = ChatOpenAI(temperature=0,model="gpt-3.5-turbo").bind(
    functions=[function_call], function_call={"name": function_call["name"]}
)
runnable = prompt | model
resp = runnable.invoke({"input": "I want the futures Gold prices"})

In [63]:
resp

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"symbol":"GC=F","provider":"yfinance"}', 'name': 'obb-derivatives-futures-historical'}}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 333, 'total_tokens': 344}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c9fd82c0-8877-43a7-aa22-ad8567f8db71-0')

In [43]:
def format_function(function_response):
    obb_func = function_response.additional_kwargs["function_call"]
    obb_func_name = obb_func["name"]
    obb_func_name = ".".join(obb_func_name.split("-")) + "("
    args_dict = ast.literal_eval(obb_func["arguments"])
    for arg, val in args_dict.items():
        if isinstance(val,str):
            val = f"'{val}'"
        obb_func_name += f"{arg}={val},"
    obb_func_name = obb_func_name[:-1] + ")"
    final_func = "from openbb import obb\n" + obb_func_name
    return final_func

obb_func = format_function(resp)

In [45]:
python_func = f"""
```python
{obb_func}
```
"""
print(python_func)


```python
from openbb import obb
obb.derivatives.futures.historical(symbol='GC=F',provider='yfinance')
```



In [49]:
obb_func = "from openbb import obb\nobb.derivatives.futures.historical(symbol='GC',provider='yfinance')"

In [50]:
from pathlib import Path

from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor
obb.user.preferences.output_type = "dataframe"

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

executor = LocalCommandLineCodeExecutor(work_dir=work_dir)
print(
    executor.execute_code_blocks(
        code_blocks=[
            CodeBlock(language="python", code=obb_func),
        ]
    )
)

exit_code=0 output='' code_file='/home/athekunal/Function Calling/hierarchical-openbb-agent/coding/tmp_code_6e9cc7fb0efbd8f982215483c4db8195.py'


In [48]:
obb.user.preferences.output_type = "dataframe"


,open,high,low,close,volume
date,,,,,
2000-08-30,273.899994,273.899994,273.899994,273.899994,0.0
2000-08-31,274.799988,278.299988,274.799988,278.299988,0.0
2000-09-01,277.000000,277.000000,277.000000,277.000000,0.0
2000-09-05,275.799988,275.799988,275.799988,275.799988,2.0
2000-09-06,274.200012,274.200012,274.200012,274.200012,0.0
...,...,...,...,...,...
2024-05-29,2340.300049,2340.300049,2340.300049,2340.300049,119317.0
2024-05-30,2336.899902,2349.500000,2320.800049,2342.899902,32587.0
2024-05-31,2344.100098,2354.000000,2319.000000,2322.899902,941.0


In [59]:
code_block = CodeBlock(language="python", code=obb_func)

In [1]:
from autogen.coding import CodeBlock, CodeExecutor, CodeExtractor, CodeResult, MarkdownCodeExtractor
from typing import List

from IPython import get_ipython

class NotebookExecutor(CodeExecutor):

    @property
    def code_extractor(self) -> CodeExtractor:
        # Extact code from markdown blocks.
        return MarkdownCodeExtractor()

    def __init__(self) -> None:
        # Get the current IPython instance running in this notebook.
        self._ipython = get_ipython()

    def execute_code_blocks(self, code_blocks: List[CodeBlock]) -> CodeResult:
        log = ""
        for code_block in code_blocks:
            result = self._ipython.run_cell("%%capture --no-display cap\n" + code_block.code)
            log += self._ipython.ev("cap.stdout")
            log += self._ipython.ev("cap.stderr")
            if result.result is not None:
                log += str(result.result)
            exitcode = 0 if result.success else 1
            if result.error_before_exec is not None:
                log += f"\n{result.error_before_exec}"
                exitcode = 1
            if result.error_in_exec is not None:
                log += f"\n{result.error_in_exec}"
                exitcode = 1
            if exitcode != 0:
                break
        return CodeResult(exit_code=exitcode, output=log)


In [7]:
import ast
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv,find_dotenv
from agent.database import load_database
from agent.dspy_agent import OpenBBAgentChroma
import os
from openbb import obb

openbb_collection = load_database(os.environ['OPENAI_API_KEY']) 

obb_chroma = OpenBBAgentChroma(openbb_collection)
obb.account.login(pat=os.environ["OBB_PAT"])
obb.user.preferences.output_type = "dataframe"
def format_function(function_response):
    obb_func = function_response.additional_kwargs["function_call"]
    obb_func_name = obb_func["name"]
    obb_func_name = ".".join(obb_func_name.split("-")) + "("
    args_dict = ast.literal_eval(obb_func["arguments"])
    for arg, val in args_dict.items():
        if isinstance(val,str):
            val = f"'{val}'"
        obb_func_name += f"{arg}={val},"
    obb_func_name = obb_func_name[:-1] + ")"
    final_func = "from openbb import obb\n" + obb_func_name
    return final_func

def main_function_calling(question:str,provider_list:List[str]=[]):
    funcs,prompts = obb_chroma(question)
    print(funcs)
    provider_sources = [fn['provider_source'] for fn in funcs[0]['metadatas']]
    if provider_list != []:
        valid_providers = [sources for sources in provider_sources if sources in provider_list]
    else:
        valid_providers = provider_sources
    for func in funcs:
        if func != []:
            for meta in func["metadatas"]:
                if meta["provider_source"] == valid_providers[0]:
                    function_call = ast.literal_eval(meta["function_call"])
                    function_call["name"] = function_call["name"].rpartition("_")[0]
                    break
    
    prompt = ChatPromptTemplate.from_messages([("human", "{input}"),("system","You can write functions from the given tool. Double check your response with correct parameter names and values. Also, check for any invalid parameter values")])

    model = ChatOpenAI(temperature=0,model="gpt-3.5-turbo").bind(
        functions=[function_call], function_call={"name": function_call["name"]}
    )
    runnable = prompt | model
    resp = runnable.invoke({"input": question})

    obb_func = format_function(resp)
    code_block = CodeBlock(language="python", code=obb_func)
    return code_block

notebook_executor = NotebookExecutor()

In [3]:
code_block = main_function_calling("What is the historical price of Ethereum?")

First level string: currency: Currency Historical Price. Currency historical data.  Currency historical prices refer to the past exchange rates of one currency against another over a specific period. This data provides insight into the fluctuations and trends in the foreign exchange market, helping analysts, traders, and economists understand currency performance, evaluate economic health, and make predictions about future movements. Currency Search.  Search available currency pairs. Currency pairs are the national currencies from two countries coupled for trading on the foreign exchange (FX) marketplace. Both currencies will have exchange rates on which the trade will have its position basis. All trading within the forex market, whether selling, buying, or trading, will take place through currency pairs. (ref: Investopedia) Major currency pairs include pairs such as EUR/USD, USD/JPY, GBP/USD, etc. Snapshots of currency exchange rates from an indirect or direct perspective of a base cu

In [4]:
code_block

CodeBlock(code="from openbb import obb\nobb.crypto.price.historical(symbol='ETH-USD',provider='fmp')", language='python')

In [6]:

notebook_executor.execute_code_blocks([code_block])

,open,high,low,close,volume,vwap,adj_close,change,change_percent
date,,,,,,,,,
2023-06-05,1890.2200,1891.34,1777.00,1811.21,9.246593e+09,1833.32,1811.82837,-79.01,-0.041800
2023-06-06,1811.9000,1898.60,1796.35,1884.95,8.704412e+09,1845.93,1884.49487,73.05,0.040300
2023-06-07,1884.9300,1898.00,1820.00,1833.60,7.919894e+09,1854.25,1832.39600,-51.33,-0.027200
2023-06-08,1832.4300,1862.70,1828.63,1846.00,4.536042e+09,1846.43,1846.30188,13.57,0.007405
2023-06-09,1847.5000,1856.00,1826.92,1840.44,4.610832e+09,1840.94,1840.22522,-7.06,-0.003821
...,...,...,...,...,...,...,...,...,...
2024-06-01,3758.3700,3833.70,3748.31,3812.48,8.661025e+09,3794.24,3813.19898,54.11,0.014400
2024-06-02,3812.8400,3837.90,3750.59,3778.88,1.112690e+10,3792.00,3780.89600,-33.96,-0.008907
2024-06-03,3778.7000,3852.10,3757.32,3766.09,1.408245e+10,3798.55,3766.38916,-12.61,-0.003337


CodeResult(exit_code=0, output='')

In [8]:
code_block = main_function_calling("What is the Gold futures price?")
print(code_block)

notebook_executor.execute_code_blocks([code_block])

First level string: derivatives: Get the complete options chain for a ticker. Historical futures prices. Futures Term Structure, current or historical. 

currency: Currency Historical Price. Currency historical data.  Currency historical prices refer to the past exchange rates of one currency against another over a specific period. This data provides insight into the fluctuations and trends in the foreign exchange market, helping analysts, traders, and economists understand currency performance, evaluate economic health, and make predictions about future movements. Currency Search.  Search available currency pairs. Currency pairs are the national currencies from two countries coupled for trading on the foreign exchange (FX) marketplace. Both currencies will have exchange rates on which the trade will have its position basis. All trading within the forex market, whether selling, buying, or trading, will take place through currency pairs. (ref: Investopedia) Major currency pairs include 

,open,high,low,close,volume
date,,,,,
2000-08-30,273.899994,273.899994,273.899994,273.899994,0.0
2000-08-31,274.799988,278.299988,274.799988,278.299988,0.0
2000-09-01,277.000000,277.000000,277.000000,277.000000,0.0
2000-09-05,275.799988,275.799988,275.799988,275.799988,2.0
2000-09-06,274.200012,274.200012,274.200012,274.200012,0.0
...,...,...,...,...,...
2024-05-30,2336.899902,2349.500000,2320.800049,2342.899902,32587.0
2024-05-31,2344.100098,2354.000000,2319.000000,2322.899902,941.0
2024-06-03,2322.600098,2347.500000,2312.000000,2346.600098,781.0


CodeResult(exit_code=0, output='')

In [43]:
code_block = main_function_calling("I want the quarterly CPI of US for the year 2024?")
print(code_block)

notebook_executor.execute_code_blocks([code_block])

First level string: economy: Get Forecasted GDP Data. Get Nominal GDP Data. Get Real GDP Data. Get the upcoming, or historical, economic calendar of global events. Get Consumer Price Index (CPI).  Returns either the rescaled index value, or a rate of change (inflation). Get Market Risk Premium by country. Search for FRED series or economic releases by ID or string.  This does not return the observation values, only the metadata. Use this function to find series IDs for `fred_series()`. Get data by series ID from FRED. Get Money Measures (M1/M2 and components).  The Federal Reserve publishes as part of the H.6 Release. Get global unemployment data. Use the composite leading indicator (CLI).  It is designed to provide early signals of turning points in business cycles showing fluctuation of the economic activity around its long term potential level.  CLIs show short-term economic movements in qualitative rather than quantitative terms. Get Short-term interest rates.  They are the rates a

OpenBBError: 
[Error] -> 1 validations error(s)
[Arg] country -> input: US -> Value error, 'us' is not available.

code="from openbb import obb\nobb.economy.cpi(country='US',units='index_2015',frequency='quarter',start_date='2024-01-01',end_date='2024-12-31',provider='fred')" language='python'


CodeResult(exit_code=0, output='')

In [61]:
print(code_block.code)

from openbb import obb
obb.equity.price.nbbo(symbol='AAPL',provider='polygon',limit=5)


In [62]:
from openbb import obb
try:
    # obb.economy.cpi(country='US',units='index_2015',frequency='quarter',start_date='2024-01-01',end_date='2024-12-31',provider='fred')
    notebook_executor.execute_code_blocks([code_block])
except Exception as e:
    error = e
    print(error)

OpenBBError: 
[Error] -> Unexpected error.

In [60]:
error

openbb_core.app.model.abstract.error.OpenBBError("\n[Error] -> 1 validations error(s)\n[Arg] country -> input: US -> Value error, 'us' is not available.")

In [11]:
try:
    notebook_executor.execute_code_blocks([code_block])
except Exception as e:
    print(e)

OpenBBError: 
[Error] -> 1 validations error(s)
[Arg] country -> input: US -> Value error, 'us' is not available.

In [52]:
code_block = main_function_calling("I want the last 5 national bid offer for Apple?")
print(code_block)

notebook_executor.execute_code_blocks([code_block])

First level string: fixedincome: Treasury Constant Maturity.  Get data for 10-Year Treasury Constant Maturity Minus Selected Treasury Constant Maturity. Constant maturity is the theoretical value of a U.S. Treasury that is based on recent values of auctioned U.S. Treasuries. The value is obtained by the U.S. Treasury on a daily basis through interpolation of the Treasury yield curve which, in turn, is based on closing bid-yields of actively-traded Treasury securities.Select Treasury Constant Maturity.  Get data for Selected Treasury Constant Maturity Minus Federal Funds Rate Constant maturity is the theoretical value of a U.S. Treasury that is based on recent values of auctioned U.S. Treasuries. The value is obtained by the U.S. Treasury on a daily basis through interpolation of the Treasury yield curve which, in turn, is based on closing bid-yields of actively-traded Treasury securities. Select Treasury Bill.  Get Selected Treasury Bill Minus Federal Funds Rate. Constant maturity is t

OpenBBError: 
[Error] -> Unexpected error.

CodeResult(exit_code=0, output='')

In [17]:
print(code_block.code)

from openbb import obb
obb.equity.price.nbbo(symbol='AAPL',provider='polygon',limit=5)


In [18]:
from openbb import obb
obb.equity.price.nbbo(symbol='AAPL',provider='polygon',limit=5)

OpenBBError: 
[Error] -> Unexpected error.

In [19]:
code_block = main_function_calling("I want historical market prices of ETF QQQ?")
print(code_block)

notebook_executor.execute_code_blocks([code_block])

First level string: etf: Search for ETFs.  An empty query returns the full list of ETFs from the provider. ETF Historical Market Price. ETF Information Overview. ETF Sector weighting. ETF Country weighting. Price performance as a return, over different periods. Get the holdings for an individual ETF. Use this function to get the holdings dates, if available. Get the recent price performance of each ticker held in the ETF. Get the exposure to ETFs for a specific stock. 

equity: Get historical and upcoming initial public offerings (IPOs). Get historical and upcoming dividend payments. Includes dividend amount, ex-dividend and payment dates. Get historical and upcoming stock split operations. Get historical and upcoming company earnings releases. Includes earnings per share (EPS) and revenue data. Get the closest peers for a given company.  Peers consist of companies trading on the same exchange, operating within the same sector and with comparable market capitalizations. Get analyst pri

,open,high,low,close,volume,vwap,adj_close,unadjusted_volume,change,change_percent
date,,,,,,,,,,
2023-06-05,354.43,357.5000,353.8500,354.90,47266228,355.1700,352.54,47266228.0,0.47,0.001326
2023-06-06,354.28,355.8200,352.9300,354.84,41390400,354.4675,352.48,41390400.0,0.56,0.001581
2023-06-07,355.13,357.1200,348.1800,348.82,69225936,352.3125,346.50,69225936.0,-6.31,-0.017800
2023-06-08,349.12,353.6200,348.8900,353.15,47153922,351.1950,350.81,47153922.0,4.03,0.011500
2023-06-09,354.63,357.6600,353.0300,354.50,53196927,354.9550,352.15,53196927.0,-0.13,-0.000367
...,...,...,...,...,...,...,...,...,...,...
2024-05-30,455.50,455.6400,450.2100,451.55,30714121,453.2250,451.55,30714121.0,-3.95,-0.008672
2024-05-31,451.78,452.4800,443.0600,450.71,55941301,449.5075,450.71,55941301.0,-1.07,-0.002368
2024-06-03,454.57,455.5800,447.9000,453.13,32997467,452.7950,453.13,32997467.0,-1.44,-0.003168


CodeResult(exit_code=0, output='')

In [26]:
code_block = main_function_calling("I want information regarding Microsoft company?")
print(code_block)

notebook_executor.execute_code_blocks([code_block])

First level string: news: World News. Global news data. Company News. Get news for one or more companies. 

equity: Get historical and upcoming initial public offerings (IPOs). Get historical and upcoming dividend payments. Includes dividend amount, ex-dividend and payment dates. Get historical and upcoming stock split operations. Get historical and upcoming company earnings releases. Includes earnings per share (EPS) and revenue data. Get the closest peers for a given company.  Peers consist of companies trading on the same exchange, operating within the same sector and with comparable market capitalizations. Get analyst price targets by company. Get historical analyst estimates for earnings and revenue. Get consensus price target and recommendation. Search for specific analysts and get their forecast track record. Get forward sales estimates. Get forward EPS estimates. Get the top price gainers in the stock market. Get the top price losers in the stock market. Get the most actively t

,symbol,name,cik,cusip,isin,stock_exchange,long_description,ceo,company_url,business_phone_no,hq_address1,hq_address_city,hq_address_postal_code,hq_state,hq_country,employees,sector,industry_category,first_stock_price_date,is_etf,is_actively_trading,is_adr,is_fund,image,currency,market_cap,last_price,year_high,year_low,volume_avg,annualized_dividend_amount,beta
0,MSFT,Microsoft Corporation,0000789019,594918104,US5949181045,NASDAQ Global Select,"Microsoft Corporation develops, licenses, and ...",Mr. Satya Nadella,https://www.microsoft.com,425 882 8080,One Microsoft Way,Redmond,98052-6399,WA,US,221000,Technology,Software - Infrastructure,1986-03-13,False,True,False,False,https://financialmodelingprep.com/image-stock/...,USD,3142343506450,422.795,433.6,309.45,19836049,3.0,0.893


code="from openbb import obb\nobb.equity.profile(symbol='MSFT',provider='fmp')" language='python'


CodeResult(exit_code=0, output='')

In [27]:
code_block = main_function_calling("I want the employee count of Alphabet?")
print(code_block)

notebook_executor.execute_code_blocks([code_block])

First level string: equity: Get historical and upcoming initial public offerings (IPOs). Get historical and upcoming dividend payments. Includes dividend amount, ex-dividend and payment dates. Get historical and upcoming stock split operations. Get historical and upcoming company earnings releases. Includes earnings per share (EPS) and revenue data. Get the closest peers for a given company.  Peers consist of companies trading on the same exchange, operating within the same sector and with comparable market capitalizations. Get analyst price targets by company. Get historical analyst estimates for earnings and revenue. Get consensus price target and recommendation. Search for specific analysts and get their forecast track record. Get forward sales estimates. Get forward EPS estimates. Get the top price gainers in the stock market. Get the top price losers in the stock market. Get the most actively traded stocks based on volume. Get potentially undervalued large cap stocks. Get potentia

OpenBBError: 
[Error] -> Unexpected error.

CodeResult(exit_code=0, output='')

In [23]:
print(code_block.code)

from openbb import obb
obb.equity.fundamental.employee_count(symbol='GOOGL',provider='fmp')


In [25]:
from openbb import obb

obb.equity.fundamental.employee_count(symbol='AAPL', provider='fmp')

OpenBBError: 
[Error] -> Unexpected error.

In [28]:
code_block = main_function_calling("I want top small cap stocks based on earnings growth sorted in ascending order?")
print(code_block)

notebook_executor.execute_code_blocks([code_block])

First level string: equity: Get historical and upcoming initial public offerings (IPOs). Get historical and upcoming dividend payments. Includes dividend amount, ex-dividend and payment dates. Get historical and upcoming stock split operations. Get historical and upcoming company earnings releases. Includes earnings per share (EPS) and revenue data. Get the closest peers for a given company.  Peers consist of companies trading on the same exchange, operating within the same sector and with comparable market capitalizations. Get analyst price targets by company. Get historical analyst estimates for earnings and revenue. Get consensus price target and recommendation. Search for specific analysts and get their forecast track record. Get forward sales estimates. Get forward EPS estimates. Get the top price gainers in the stock market. Get the top price losers in the stock market. Get the most actively traded stocks based on volume. Get potentially undervalued large cap stocks. Get potentia

,symbol,name,market_cap,sector,industry,beta,price,last_annual_dividend,volume,exchange,exchange_name,country,is_etf,actively_trading,isFund
0,FFFAX,Fidelity Freedom Income Fund,1999594999,Financial Services,Asset Management - Income,0.560,10.560,0.263,NaN,NASDAQ,NASDAQ,US,False,True,True
1,FLIFX,Fidelity Freedom Index 2015 Fund - Investor Class,1998600315,Financial Services,Asset Management,0.800,14.290,0.335,NaN,NASDAQ,NASDAQ,US,False,True,True
2,HTH,Hilltop Holdings Inc.,1998472478,Financial Services,Banks - Regional,1.103,30.620,0.680,95178.0,NYSE,New York Stock Exchange,US,False,True,False
3,ARG.PA,Argan SA,1998336234,Real Estate,REIT - Industrial,0.735,78.700,3.150,7215.0,EURONEXT,Paris,FR,False,True,False
4,FTANX,Fidelity Asset Manager 30%,1998330871,,,0.670,11.690,0.274,NaN,NASDAQ,NASDAQ,US,False,True,True
5,BMCIX,BlackRock High Equity Income Fund,1998070644,Financial Services,Asset Management - Income,0.720,28.840,1.221,NaN,NASDAQ,NASDAQ,None,False,True,True
6,TVIIX,TIAA-CREF Lifecycle Index 2060 Instl,1996825201,,,1.270,18.910,0.356,NaN,NASDAQ,Nasdaq,None,False,True,True
7,BRDG,Bridge Investment Group Holdings Inc.,1996322114,Financial Services,Asset Management,1.596,7.695,0.480,66342.0,NYSE,New York Stock Exchange,US,False,True,False
8,STAA,STAAR Surgical Company,1995705282,Healthcare,Medical - Instruments & Supplies,0.806,40.620,NaN,130342.0,NASDAQ,NASDAQ Global Market,US,False,True,False
9,UFPT,"UFP Technologies, Inc.",1994020580,Healthcare,Medical - Devices,0.897,259.960,NaN,15578.0,NASDAQ,NASDAQ Capital Market,US,False,True,False


CodeResult(exit_code=0, output='')

In [29]:
code_block = main_function_calling("I want undervalued growth stocks sorted in ascending order?")
print(code_block)

notebook_executor.execute_code_blocks([code_block])

First level string: equity: Get historical and upcoming initial public offerings (IPOs). Get historical and upcoming dividend payments. Includes dividend amount, ex-dividend and payment dates. Get historical and upcoming stock split operations. Get historical and upcoming company earnings releases. Includes earnings per share (EPS) and revenue data. Get the closest peers for a given company.  Peers consist of companies trading on the same exchange, operating within the same sector and with comparable market capitalizations. Get analyst price targets by company. Get historical analyst estimates for earnings and revenue. Get consensus price target and recommendation. Search for specific analysts and get their forecast track record. Get forward sales estimates. Get forward EPS estimates. Get the top price gainers in the stock market. Get the top price losers in the stock market. Get the most actively traded stocks based on volume. Get potentially undervalued large cap stocks. Get potentia

,symbol,name,price,change,percent_change,volume,market_cap,avg_volume_3_months,pe_ratio_ttm
0,BRK-B,Berkshire Hathaway Inc.,408.2200,-1.3100,-0.32,1477000.0,8.815470e+11,3193000.0,12.04
1,IBN,ICICI Bank Limited,26.5300,0.7600,2.97,4175000.0,9.336200e+10,4639000.0,17.81
2,CVS,CVS Health Corporation,60.5300,0.2800,0.46,5885000.0,7.599400e+10,12123000.0,10.62
3,TGT,Target Corporation,147.3800,-3.1700,-2.11,2980000.0,6.818300e+10,3828000.0,16.54
4,PYPL,"PayPal Holdings, Inc.",63.3100,-0.0200,-0.03,5225000.0,6.622500e+10,11926000.0,15.95
5,FDX,FedEx Corporation,247.2500,1.3000,0.53,457345.0,6.084400e+10,1700000.0,14.26
6,BBVA,"Banco Bilbao Vizcaya Argentaria, S.A.",10.4300,0.1200,1.21,702247.0,6.015100e+10,1913000.0,7.04
7,GM,General Motors Company,45.5400,0.2800,0.63,4676000.0,5.193900e+10,14772000.0,5.56
8,F,Ford Motor Company,12.0100,-0.0200,-0.17,29220000.0,4.772300e+10,48789000.0,12.38
9,ACGL,Arch Capital Group Ltd.,99.6000,-1.2600,-1.25,615240.0,3.739700e+10,1822000.0,7.86


CodeResult(exit_code=0, output='/home/athekunal/Function Calling/openbb-env/lib/python3.10/site-packages/openbb_yfinance/models/undervalued_growth_equities.py:82: FutureWarning: Passing literal html to \'read_html\' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a \'StringIO\' object.\n  df = pd.read_html(html_clean, header=None)[0].dropna(how="all", axis=1)\n')

In [30]:
code_block = main_function_calling("I want the forward sales estiates for Apple for the fisal period Q1?")
print(code_block)

notebook_executor.execute_code_blocks([code_block])

First level string: equity: Get historical and upcoming initial public offerings (IPOs). Get historical and upcoming dividend payments. Includes dividend amount, ex-dividend and payment dates. Get historical and upcoming stock split operations. Get historical and upcoming company earnings releases. Includes earnings per share (EPS) and revenue data. Get the closest peers for a given company.  Peers consist of companies trading on the same exchange, operating within the same sector and with comparable market capitalizations. Get analyst price targets by company. Get historical analyst estimates for earnings and revenue. Get consensus price target and recommendation. Search for specific analysts and get their forecast track record. Get forward sales estimates. Get forward EPS estimates. Get the top price gainers in the stock market. Get the top price losers in the stock market. Get the most actively traded stocks based on volume. Get potentially undervalued large cap stocks. Get potentia

OpenBBError: 
[Error] -> Unexpected error.

CodeResult(exit_code=0, output='')

In [31]:
code_block = main_function_calling("I want to find upcoming IPOs with minimum value of $1M?")
print(code_block)

notebook_executor.execute_code_blocks([code_block])

First level string: equity: Get historical and upcoming initial public offerings (IPOs). Get historical and upcoming dividend payments. Includes dividend amount, ex-dividend and payment dates. Get historical and upcoming stock split operations. Get historical and upcoming company earnings releases. Includes earnings per share (EPS) and revenue data. Get the closest peers for a given company.  Peers consist of companies trading on the same exchange, operating within the same sector and with comparable market capitalizations. Get analyst price targets by company. Get historical analyst estimates for earnings and revenue. Get consensus price target and recommendation. Search for specific analysts and get their forecast track record. Get forward sales estimates. Get forward EPS estimates. Get the top price gainers in the stock market. Get the top price losers in the stock market. Get the most actively traded stocks based on volume. Get potentially undervalued large cap stocks. Get potentia

OpenBBError: 
[Error] -> Unexpected error.

code="from openbb import obb\nobb.equity.calendar.ipo(min_value=1000000,status='upcoming',provider='intrinio')" language='python'


CodeResult(exit_code=0, output='')

In [34]:
code_block = main_function_calling("I want the nominal GDP data of Australia capped to USD?")
print(code_block)

notebook_executor.execute_code_blocks([code_block])

First level string: economy: Get Forecasted GDP Data. Get Nominal GDP Data. Get Real GDP Data. Get the upcoming, or historical, economic calendar of global events. Get Consumer Price Index (CPI).  Returns either the rescaled index value, or a rate of change (inflation). Get Market Risk Premium by country. Search for FRED series or economic releases by ID or string.  This does not return the observation values, only the metadata. Use this function to find series IDs for `fred_series()`. Get data by series ID from FRED. Get Money Measures (M1/M2 and components).  The Federal Reserve publishes as part of the H.6 Release. Get global unemployment data. Use the composite leading indicator (CLI).  It is designed to provide early signals of turning points in business cycles showing fluctuation of the economic activity around its long term potential level.  CLIs show short-term economic movements in qualitative rather than quantitative terms. Get Short-term interest rates.  They are the rates a

,value,country
date,,
1960-12-31,2412.627589,australia
1961-12-31,2383.188902,australia
1962-12-31,2577.332834,australia
1963-12-31,2752.620592,australia
1964-12-31,2902.590472,australia
...,...,...
2018-12-31,53118.792237,australia
2019-12-31,53670.040070,australia
2020-12-31,56888.304477,australia


CodeResult(exit_code=0, output='')

In [35]:
code_block

CodeBlock(code="from openbb import obb\nobb.economy.gdp.nominal(units='usd_cap',provider='oecd',country='australia')", language='python')

In [36]:
code_block = main_function_calling("I want the current futures term structure for NG for 14 April 2024?")
print(code_block)

notebook_executor.execute_code_blocks([code_block])

First level string: derivatives: Get the complete options chain for a ticker. Historical futures prices. Futures Term Structure, current or historical. 

fixedincome: Treasury Constant Maturity.  Get data for 10-Year Treasury Constant Maturity Minus Selected Treasury Constant Maturity. Constant maturity is the theoretical value of a U.S. Treasury that is based on recent values of auctioned U.S. Treasuries. The value is obtained by the U.S. Treasury on a daily basis through interpolation of the Treasury yield curve which, in turn, is based on closing bid-yields of actively-traded Treasury securities.Select Treasury Constant Maturity.  Get data for Selected Treasury Constant Maturity Minus Federal Funds Rate Constant maturity is the theoretical value of a U.S. Treasury that is based on recent values of auctioned U.S. Treasuries. The value is obtained by the U.S. Treasury on a daily basis through interpolation of the Treasury yield curve which, in turn, is based on closing bid-yields of a

,expiration
0,Jul-2024
1,Aug-2024
2,Sep-2024
3,Oct-2024
4,Nov-2024
...,...
73,Aug-2030
74,Sep-2030
75,Oct-2030
76,Nov-2030


CodeResult(exit_code=0, output='')

In [37]:
code_block = main_function_calling("I want the quarterly unemployment data for female who ar ebetween 15-24 of age in Australia with seasonal adjustment?")
print(code_block)

notebook_executor.execute_code_blocks([code_block])

First level string: economy: Get Forecasted GDP Data. Get Nominal GDP Data. Get Real GDP Data. Get the upcoming, or historical, economic calendar of global events. Get Consumer Price Index (CPI).  Returns either the rescaled index value, or a rate of change (inflation). Get Market Risk Premium by country. Search for FRED series or economic releases by ID or string.  This does not return the observation values, only the metadata. Use this function to find series IDs for `fred_series()`. Get data by series ID from FRED. Get Money Measures (M1/M2 and components).  The Federal Reserve publishes as part of the H.6 Release. Get global unemployment data. Use the composite leading indicator (CLI).  It is designed to provide early signals of turning points in business cycles showing fluctuation of the economic activity around its long term potential level.  CLIs show short-term economic movements in qualitative rather than quantitative terms. Get Short-term interest rates.  They are the rates a

,value,country
date,,
1978-06-30,13.260880,australia
1978-09-30,13.502720,australia
1978-12-31,14.095050,australia
1979-03-31,14.079530,australia
1979-06-30,14.693110,australia
...,...,...
2023-03-31,7.086318,australia
2023-06-30,6.832390,australia
2023-09-30,7.636237,australia


CodeResult(exit_code=0, output='')

In [38]:
code_block = main_function_calling("I want the market risk premium by country?")
print(code_block)

notebook_executor.execute_code_blocks([code_block])

First level string: fixedincome: Treasury Constant Maturity.  Get data for 10-Year Treasury Constant Maturity Minus Selected Treasury Constant Maturity. Constant maturity is the theoretical value of a U.S. Treasury that is based on recent values of auctioned U.S. Treasuries. The value is obtained by the U.S. Treasury on a daily basis through interpolation of the Treasury yield curve which, in turn, is based on closing bid-yields of actively-traded Treasury securities.Select Treasury Constant Maturity.  Get data for Selected Treasury Constant Maturity Minus Federal Funds Rate Constant maturity is the theoretical value of a U.S. Treasury that is based on recent values of auctioned U.S. Treasuries. The value is obtained by the U.S. Treasury on a daily basis through interpolation of the Treasury yield curve which, in turn, is based on closing bid-yields of actively-traded Treasury securities. Select Treasury Bill.  Get Selected Treasury Bill Minus Federal Funds Rate. Constant maturity is t

OpenBBError: 
[Error] -> Unexpected error.

CodeResult(exit_code=0, output='')

In [39]:
code_block = main_function_calling("I want the cryptocurrency pair of Bitcoin?")
print(code_block)

notebook_executor.execute_code_blocks([code_block])

First level string: crypto: Get historical price data for cryptocurrency pair(s) within a provider. Search available cryptocurrency pairs within a provider. 

currency: Currency Historical Price. Currency historical data.  Currency historical prices refer to the past exchange rates of one currency against another over a specific period. This data provides insight into the fluctuations and trends in the foreign exchange market, helping analysts, traders, and economists understand currency performance, evaluate economic health, and make predictions about future movements. Currency Search.  Search available currency pairs. Currency pairs are the national currencies from two countries coupled for trading on the foreign exchange (FX) marketplace. Both currencies will have exchange rates on which the trade will have its position basis. All trading within the forex market, whether selling, buying, or trading, will take place through currency pairs. (ref: Investopedia) Major currency pairs inc

,symbol,name,currency,exchange,exchange_name
0,NMBTCUSD,NanoMeter Bitcoin USD,USD,CCC,CRYPTO
1,BCHUSD,Bitcoin Cash USD,USD,CCC,CRYPTO
2,BITGUSD,Bitcoin Green USD,USD,CCC,CRYPTO
3,BTCDUSD,BitcoinDark USD,USD,CCC,CRYPTO
4,BSVUSD,Bitcoin SV USD,USD,CCC,CRYPTO
5,BBTCUSD,Baby Bitcoin USD,USD,CCC,CRYPTO
6,BTC.BUSD,Bitcoin Avalanche Bridged USD,USD,CCC,CRYPTO
7,IMBTCUSD,The Tokenized Bitcoin USD,USD,CCC,CRYPTO
8,BCUSD,Bitcoin Confidential USD,USD,CCC,CRYPTO
9,ORBUSD,Orbitcoin USD,USD,CCC,CRYPTO


CodeResult(exit_code=0, output='')

In [41]:
code_block = main_function_calling("I want the consensus price target and recommendation for Apple?",['yfinance'])
print(code_block)

notebook_executor.execute_code_blocks([code_block])

First level string: equity: Get historical and upcoming initial public offerings (IPOs). Get historical and upcoming dividend payments. Includes dividend amount, ex-dividend and payment dates. Get historical and upcoming stock split operations. Get historical and upcoming company earnings releases. Includes earnings per share (EPS) and revenue data. Get the closest peers for a given company.  Peers consist of companies trading on the same exchange, operating within the same sector and with comparable market capitalizations. Get analyst price targets by company. Get historical analyst estimates for earnings and revenue. Get consensus price target and recommendation. Search for specific analysts and get their forecast track record. Get forward sales estimates. Get forward EPS estimates. Get the top price gainers in the stock market. Get the top price losers in the stock market. Get the most actively traded stocks based on volume. Get potentially undervalued large cap stocks. Get potentia

,symbol,target_high,target_low,target_consensus,target_median,recommendation,recommendation_mean,number_of_analysts,current_price,currency
0,AAPL,275.0,164.0,204.31,200.0,buy,2.1,39,196.33,USD


CodeResult(exit_code=0, output='')

In [42]:
code_block = main_function_calling("I want the consensus price target and recommendation for Microsoft?",['yfinance'])
print(code_block)

notebook_executor.execute_code_blocks([code_block])

First level string: equity: Get historical and upcoming initial public offerings (IPOs). Get historical and upcoming dividend payments. Includes dividend amount, ex-dividend and payment dates. Get historical and upcoming stock split operations. Get historical and upcoming company earnings releases. Includes earnings per share (EPS) and revenue data. Get the closest peers for a given company.  Peers consist of companies trading on the same exchange, operating within the same sector and with comparable market capitalizations. Get analyst price targets by company. Get historical analyst estimates for earnings and revenue. Get consensus price target and recommendation. Search for specific analysts and get their forecast track record. Get forward sales estimates. Get forward EPS estimates. Get the top price gainers in the stock market. Get the top price losers in the stock market. Get the most actively traded stocks based on volume. Get potentially undervalued large cap stocks. Get potentia

,symbol,target_high,target_low,target_consensus,target_median,recommendation,recommendation_mean,number_of_analysts,current_price,currency
0,MSFT,600.0,436.0,483.33,477.33,buy,1.7,47,423.16,USD


CodeResult(exit_code=0, output='')

In [69]:
import dspy

def format_function(function_response):
    obb_func = function_response.additional_kwargs["function_call"]
    obb_func_name = obb_func["name"]
    obb_func_name = ".".join(obb_func_name.split("-")) + "("
    args_dict = ast.literal_eval(obb_func["arguments"])
    for arg, val in args_dict.items():
        if isinstance(val,str):
            val = f"'{val}'"
        obb_func_name += f"{arg}={val},"
    obb_func_name = obb_func_name[:-1] + ")"
    final_func = "from openbb import obb\n" + obb_func_name
    return final_func

class DSPYOpenBBAgent(dspy.Module):
    def __init__(self,obb_hierarchical_agent):
        self.obb_hierarchical_agent = obb_hierarchical_agent
        self.notebook_executor = NotebookExecutor()
        self.langchain_model = ChatOpenAI(temperature=0,model="gpt-3.5-turbo")
        
    def __call__(self, question, provider_list:List[str]=[],**kwargs):
        return super().__call__(question, provider_list,**kwargs)
    
    def forward(self,question,provider_list:List[str]=[]):
        funcs,_ = obb_chroma(question)
        print(funcs)
        provider_sources = [fn['provider_source'] for fn in funcs[0]['metadatas']]
        if provider_list != []:
            valid_providers = [sources for sources in provider_sources if sources in provider_list]
        else:
            valid_providers = provider_sources
        
        for vp in valid_providers:
            for func in funcs:
                if func != []:
                    for meta in func["metadatas"]:
                        if meta["provider_source"] == vp:
                            function_call = ast.literal_eval(meta["function_call"])
                            function_call["name"] = function_call["name"].rpartition("_")[0]
                            break
            max_tries = 0
            system_message = "You can write functions from the given tool. Double check your response with correct parameter names and values. Also, check for any invalid parameter values"
            while True:
                if max_tries>3: 
                    print(f"\033[31mCouldn't resolve the error {e} with the code {code_block.code}\033[0m")
                    break
                model = self.langchain_model.bind(
                    functions=[function_call], function_call={"name": function_call["name"]}
                )
                prompt = ChatPromptTemplate.from_messages([("human", "{input}"),("system",system_message)])
                runnable = prompt | model   
                resp = runnable.invoke({"input": question})

                obb_func = format_function(resp)
                code_block = CodeBlock(language="python", code=obb_func)
                
                try:
                    
                    out = notebook_executor.execute_code_blocks([code_block])
                    print(out)
                    return
                except Exception as e:
                    if "Unexpected error" in e:
                        print(f"Failed with provider {vp}")
                        break
                    else:
                        
                        system_message = f"Resolve the following error {e} by writing the function from the given tool. Double check your response so that you are resolving the error"
                        max_tries += 1
                finally:
                    return 

dspy_obb = DSPYOpenBBAgent(obb_chroma)

In [71]:
dspy_obb("I want information regarding Microsoft company?")

First level string: news: World News. Global news data. Company News. Get news for one or more companies. 

equity: Get historical and upcoming initial public offerings (IPOs). Get historical and upcoming dividend payments. Includes dividend amount, ex-dividend and payment dates. Get historical and upcoming stock split operations. Get historical and upcoming company earnings releases. Includes earnings per share (EPS) and revenue data. Get the closest peers for a given company.  Peers consist of companies trading on the same exchange, operating within the same sector and with comparable market capitalizations. Get analyst price targets by company. Get historical analyst estimates for earnings and revenue. Get consensus price target and recommendation. Search for specific analysts and get their forecast track record. Get forward sales estimates. Get forward EPS estimates. Get the top price gainers in the stock market. Get the top price losers in the stock market. Get the most actively t

,symbol,name,cik,cusip,isin,stock_exchange,long_description,ceo,company_url,business_phone_no,hq_address1,hq_address_city,hq_address_postal_code,hq_state,hq_country,employees,sector,industry_category,first_stock_price_date,is_etf,is_actively_trading,is_adr,is_fund,image,currency,market_cap,last_price,year_high,year_low,volume_avg,annualized_dividend_amount,beta
0,MSFT,Microsoft Corporation,0000789019,594918104,US5949181045,NASDAQ Global Select,"Microsoft Corporation develops, licenses, and ...",Mr. Satya Nadella,https://www.microsoft.com,425 882 8080,One Microsoft Way,Redmond,98052-6399,WA,US,221000,Technology,Software - Infrastructure,1986-03-13,False,True,False,False,https://financialmodelingprep.com/image-stock/...,USD,3138738836100,422.31,433.6,309.45,19836049,3.0,0.893


exit_code=0 output=''


## USING AUTOGEN FOR IT

In [73]:
dspy_obb("What is the Copper futures price?")

First level string: derivatives: Get the complete options chain for a ticker. Historical futures prices. Futures Term Structure, current or historical. 

currency: Currency Historical Price. Currency historical data.  Currency historical prices refer to the past exchange rates of one currency against another over a specific period. This data provides insight into the fluctuations and trends in the foreign exchange market, helping analysts, traders, and economists understand currency performance, evaluate economic health, and make predictions about future movements. Currency Search.  Search available currency pairs. Currency pairs are the national currencies from two countries coupled for trading on the foreign exchange (FX) marketplace. Both currencies will have exchange rates on which the trade will have its position basis. All trading within the forex market, whether selling, buying, or trading, will take place through currency pairs. (ref: Investopedia) Major currency pairs include 

,open,high,low,close,volume
date,,,,,
2000-08-30,0.8790,0.8870,0.8770,0.8850,2886.0
2000-08-31,0.8850,0.8880,0.8800,0.8850,1095.0
2000-09-01,0.8780,0.8890,0.8780,0.8890,3449.0
2000-09-05,0.8960,0.9070,0.8950,0.9060,1397.0
2000-09-06,0.9050,0.9060,0.8975,0.9015,1195.0
...,...,...,...,...,...
2024-05-30,4.7755,4.7855,4.6425,4.6720,1170.0
2024-05-31,4.6385,4.6515,4.5835,4.6100,615.0
2024-06-03,4.6840,4.6840,4.6630,4.6770,488.0


exit_code=0 output=''


In [74]:
dspy_obb("I want the cryptocurrency pair of Ethereum?")

First level string: crypto: Get historical price data for cryptocurrency pair(s) within a provider. Search available cryptocurrency pairs within a provider. 

currency: Currency Historical Price. Currency historical data.  Currency historical prices refer to the past exchange rates of one currency against another over a specific period. This data provides insight into the fluctuations and trends in the foreign exchange market, helping analysts, traders, and economists understand currency performance, evaluate economic health, and make predictions about future movements. Currency Search.  Search available currency pairs. Currency pairs are the national currencies from two countries coupled for trading on the foreign exchange (FX) marketplace. Both currencies will have exchange rates on which the trade will have its position basis. All trading within the forex market, whether selling, buying, or trading, will take place through currency pairs. (ref: Investopedia) Major currency pairs inc

,symbol,name,currency,exchange,exchange_name
0,ETHYUSD,Ethereum Yield USD,USD,CCC,CRYPTO
1,ENSUSD,Ethereum Name Service USD,USD,CCC,CRYPTO
2,WETHWUSD,Wrapped EthereumPoW USD,USD,CCC,CRYPTO
3,ETHYSUSD,Ethereum Stake USD,USD,CCC,CRYPTO
4,AETHUSD,Aave Ethereum USD,USD,CCC,CRYPTO
5,T99USD,Tethereum USD,USD,CCC,CRYPTO
6,ETGPUSD,Ethereum Gold Project USD,USD,CCC,CRYPTO
7,EMAXUSD,EthereumMax USD,USD,CCC,CRYPTO
8,EMSUSD,Ethereum Message Search USD,USD,CCC,CRYPTO
9,ETHFUSD,EthereumFair USD,USD,CCC,CRYPTO


exit_code=0 output=''
